### Python package voor een FewsPI service

Voorbeeld notebook om vanuit Python te praten met een openbare of lokale FewsPi service.

Allereerst, importeer de module als volgt:

In [1]:
import sys
sys.path.append("..") 
from hkvfewspy.io.fewspi import pi

version 0.3.0

De volgende functies zijn beschikbaar via de module

- setClient
- getFilters
- getParameters
- getTimeSeries
- getTimeSeriesForFilter2 ! this is the 'v0.2.2 getTimeSeries'
- getTimeZoneID
- getAvailableTimeZones

Om een pi service uit te lezen moet deze eerst als client gezet worden. Dit kan met de functie `setClient`. Een wsdl service URL moet ingevuld worden en dat is bijvoorbeeld `http://localhost:8081/FewsPiService/fewspiservice?wsdl` of `http://localhost:8101/FewsPiService?wsdl`

In [2]:
pi.setClient(wsdl = 'http://localhost:8081/FewsPiService/fewspiservice?wsdl')

In [3]:
from datetime import datetime
startTime = datetime(2017,12,29)
endTime =  datetime(2018,1,1)

In [4]:
params = dict(
    moduleInstanceIds=['Boezemwaterstanden2'],
    parameterIds=['H.meting.10min'],
    startTime=startTime,
    endTime=endTime,
    clientTimeZone = 'Etc/GMT+1',
    forecastSearchCount=1,
    convertDatum='false',
    useDisplayUnits='false',
    showThresholds='true',
    omitMissing='false',
    onlyHeaders='false',
    onlyManualEdits='false',
    showStatistics='false',
    ensembleId='',
    importFromExternalDataSource='false',
    showEnsembleMemberIds='false',
    piVersion='1.23'
)

In [5]:
df, entry = pi.getTimeSeries(params, setFormat='df')
df.head()

flag  \
date                      moduleId qualifierId parameterId    units locationId   stationName                            
2017-12-29 00:00:00-01:00                      H.meting.10min m/nap 60001_boezem Winkel, Boezem                   0.0   
                                                                    60009_boezem Pondskoekersluis, Boezem         0.0   
                                                                    60010_boezem De Ruiter Demmerik, Boezempeil   0.0   
                                                                    60015_boezem Oudhuizersluis, Boezem           0.0   
                                                                    60030_boezem Derde Bedijking, Boezem          0.0   

                                                                                                                 value  
date                      moduleId qualifierId parameterId    units locationId   stationName                            
2017-12-29 00:00:00-01:00                      H.meting.10min m/nap 60001_boezem Winkel, Boezem                 -0.434  
                                                                    60009_boezem Pondskoekersluis, Boezem       -0.407  
                                                                    60010_boezem De Ruiter Demmerik, Boezempeil -0.395  
                                                                    60015_boezem Oudhuizersluis, Boezem         -0.428  
                                                                    60030_boezem Derde Bedijking, Boezem        -0.426

In [6]:
df.count()

flag     28578
value    28578
dtype: int64

In [ ]:
filters = pi.getFilters()
vars(filters)

Een uitgelezen filter welke als attribute is toegevoegd kan gebruikt worden als `filterId`.

In [ ]:
filterId = pi.Filters.boezem['id']
print (filterId)

Vervolgens kunnen de locties voor de desbetreffende filter uitgelezen worden met de `getLocations` functie. Met `setFormat` kan het formaat worden bepaald hoe het object teruggegeven moet worden. Dit kan zijn:
- `gdf` voor een GeoDataFrame formaat
- `geojson` voor een GeoJSON formaat
- `dict` voor een Dictionary formaat

(De verschillende formaten worden momenteel allemaal ook als attributes bijgeschreven onder het `pi.Locations` object)

In [ ]:
gdf = pi.getLocations(filterId = filterId, piVersion = '1.23', setFormat = 'gdf')

Met een GeoDataFrame formaat kan je het gelijk visualiseren of -gewoon- als DataFrame vervolg acties op doen

In [ ]:
%matplotlib inline
gdf.plot()

In [ ]:
gdf.head()

Om een `list` te krijgen van alle beschikbare locaties doe het volgende. Er wordt vervolgens alleen de eerste 5 items getoond

In [ ]:
locationIds = gdf.locationId.tolist()
locationIds[0:5]

Zoals gezegd worden de verschillende formaten als attributes onder het `pi.Locations` object weggeschreven, waarbij het GeoJSON formaat gelijk binnen een mapping module zoals `folium` gebruikt kan worden:

In [ ]:
# import folium
# mapa = folium.Map([53.2, 5.5],
#                   zoom_start=9,
#                   tiles='cartodbpositron')

# geoJSONdata = pi.Locations.asGeoJSON
# points = folium.features.GeoJson(pi.Locations.asGeoJSON)

# mapa.add_children(points)
# mapa

Om ook popups te linken aan folium is het makkelijker om vanuit een GeoDataFrame te werken en niet via een GeoJSON bestand

In [ ]:
import folium
mapa = folium.Map([53.2, 5.5],
                  zoom_start=9,
                  tiles='cartodbpositron')

fg=folium.FeatureGroup(name="locations")
for idx, row in gdf.iterrows():
    fg.add_child(folium.Marker(location=[row['geometry'].y, row['geometry'].x],
                              popup=(folium.Popup('NAAM: '+row['shortName']+" ID: "+row['locationId'])))) 
mapa.add_child(fg)
mapa.add_child(folium.LayerControl())
mapa

Of maak gebruik van de geojsonio module om het te parsen naar een site als www.geojson.io inclusief opmaak van tooltips

In [ ]:
#import geojsonio
#geojsonio.embed(pi.Locations.asGeoJSON) # dit is niet embed, maar doet een redirect naar de website. Save notebook eerst.

Net zoals de locaties kunnen worden uitgelezen voor een filter is dat ook zo voor de parameters. Dit kan met de `getParameters` functie:

In [ ]:
params = pi.getParameters(filterId=filterId)
vars(params)

Ook deze worden als attribute onder het object opgeslagen.

In [ ]:
parameterIds = pi.Parameters.DP_meting['id']
parameterIds

Voor de `getTimeSeries` functie zijn ook nog de start en eindtijd nodig om de periode te bepalen voor de data request. Dit kan met een `datetime` object.

In [ ]:
from datetime import datetime
startTime = datetime(2017,3,1)
endTime =  datetime.now()

In dit geval zijn de startTime en endTime niet bewust van een timezone. In dit geval worden deze intern alsnog bepaald op basis `clientTimeZone` parameter. Standaard staat deze op `Europe/Amsterdam`. Mogelijke timezones worden bepaald met de pytz module (`import pytz`, `pytz.all_timezones`). 

Er worden items terug gegeven. Eerst het data object zelf in het gewenste `format`. En daarnaast nog een `entry`, wat op dit moment een combinatie is van `parameterId|locationId|units`. Deze `entry` zou bijvoorbeeld gebruikt kunnen worden als `key` voor een entry in de hkv.services dataportal 

Zoals gezegd zijn er meerdere formats, welke met `setFormat` ingesteld kan worden. Keuze kan gemaakt worden uit:
- `df` voor pandas DataFrame
- `json` voor een JSON formatted output
- `gzip` voor een GZIP gecomprimiteerde JSON string

In [ ]:
df, entry = pi.getTimeSeriesForFilter2(filterId = filterId,
                             parameterIds = parameterIds, 
                             locationIds=locationIds, 
                             startTime=startTime, 
                             endTime=endTime, 
                             clientTimeZone='Etc/GMT+1',
                             setFormat='df')
print (entry)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
df.reset_index(inplace=True)
df['value'].replace(-999.0,np.NaN, inplace = True)

In [ ]:
fig, ax = plt.subplots()
ax.plot(df['date'],df['value'],'-')
ylabel = df['parameterId'][0] + ' ('+df['units'][0]+')'
ax.set_ylabel(ylabel)
plt.show()

In [ ]:
pi.TimeSeries.asJSON[0:1000]

In [ ]:
availableTimeZones = pi.getAvailableTimeZones()
print(availableTimeZones)